In [1]:
# load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import plotly.express as px
import plotly.graph_objects as go
import gc

In [2]:
# read data
data_all = pd.read_csv('gimnasio.csv')

In [3]:
data_all.head(10)

,Intensity_economia,Polarization_economia,Subjectivity_economia,Intensity_gimnasio,Polarization_gimnasio,gimnasio,sanitas,adeslas,decathlon,IBEX35_close,crecimiento,date
0,0.422,0.241,0.115,0.502,0.383,0.667,0.667,0.5,0.014,0.739,0.5,2014-01-13
1,0.422,0.241,0.115,0.483,0.378,0.667,0.667,0.5,0.014,0.742,0.5,2014-01-14
2,0.447,0.282,0.091,0.480,0.370,0.667,0.667,0.5,0.014,0.767,0.5,2014-01-15
3,0.409,0.283,0.050,0.488,0.363,0.667,0.667,0.5,0.014,0.755,0.5,2014-01-16
4,0.425,0.260,0.074,0.501,0.381,0.667,0.667,0.5,0.014,0.757,0.5,2014-01-17
5,0.466,0.279,0.013,0.509,0.405,0.667,0.667,0.5,0.014,0.755,0.5,2014-01-18
6,0.420,0.243,0.108,0.510,0.369,0.667,0.667,0.5,0.014,0.755,0.5,2014-01-19
7,0.425,0.263,0.067,0.476,0.367,0.667,0.667,0.5,0.014,0.755,0.5,2014-01-20
8,0.422,0.253,0.088,0.492,0.382,0.667,0.667,0.5,0.014,0.738,0.5,2014-01-21
9,0.395,0.246,0.049,0.510,0.399,0.667,0.667,0.5,0.014,0.724,0.5,2014-01-22


In [4]:
target = 'Intensity_gimnasio'

In [5]:
data=data_all[{target}].dropna()

In [6]:
# plot the data
px.scatter(x = data.index, y=data[target],
       labels={'x':'Number of time steps (days)',
               'y':f'{target}'})

In [7]:
# we transform the data by scaling each feature to a range 0÷1
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler(feature_range=(0,1))
data = scale.fit_transform(data)

In [8]:
# divide the data into number of steps 
def create_data(data, seq_len):
    N = len(data)
    X = []
    Y = []
    for i in range(N-seq_len-1):
        x = data[i:i+seq_len]
        X.append(x)
        y = data[i+seq_len]
        Y.append(y)        
    return X,Y

In [9]:
x, y = create_data(data,10)
# convert the x and y values into arrays
x, y = np.asarray(x), np.asarray(y)
x.reshape(-1,10).shape

(2440, 10)

In [10]:
# divide the data into train and test
def split_data(x,y, ratio):
    assert len(x)==len(y)
    N = len(x)
    train_x, test_x  = x[:int(N*ratio)], x[int(N*ratio):]
    train_y, test_y  = y[:int(N*ratio)], y[int(N*ratio):]
    return train_x,train_y, test_x, test_y

### Recurrent Neural Network - RNN

In [11]:
# Implement the RNN 
class Simple_RNN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, num_layers):
        super().__init__()
        
        # define the dimensions
        self.in_dim = in_dim
        self.hid_dim = in_dim
        self.out_dim = out_dim
        self.layer = num_layers
        
        # define the rnn layer
        self.rnn = nn.RNN(self.in_dim, self.hid_dim, self.layer, nonlinearity='tanh', batch_first=True)
        
        # define fully connected layer for output
        self.fc = nn.Linear(self.hid_dim, self.out_dim)
        
    def forward(self,x):
        
        # initialize the hidden layer
        h0 = torch.zeros(self.layer, x.size(0),self.hid_dim)
        
        #initialize the rnn
        out, _ = self.rnn(x,h0)
        
        out = self.fc(out[:,-1,:])
        return out

In [12]:
# convert the data from numpy to tensor
train_x, train_y, test_x, test_y = split_data(x, y, 0.7)
train_x = torch.from_numpy(train_x.astype(np.float32))
train_y = torch.from_numpy(train_y.astype(np.float32))
test_x = torch.from_numpy(test_x.astype(np.float32))
test_y = torch.from_numpy(test_y.astype(np.float32))

In [13]:
x.shape

(2440, 10, 1)

In [14]:
# Parameters of RNN neural network
in_size = x.shape[-1]
out_size = in_size
hid_size = 3
num_layers = 1

In [15]:
# _______________MODEL___________________________________
model = Simple_RNN(in_size,
                   hid_size,
                   out_size, 
                   num_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fun = nn.MSELoss()
epochs = 500

In [16]:
def train_model(model,
               criterion,
               optimizer,
               x_train,
               x_test,
               y_train,
               y_test,
               epochs=epochs):   
    train_loss = np.zeros(epochs)
    test_loss = np.zeros(epochs)    
    for epoch in range(epochs):        
        optimizer.zero_grad() # put default model grads to zero
        
        # predict the output
        pred = model(x_train)
        
        # calculate the loss 
        error = criterion(pred,y_train)
        
        # backpropagate the error
        error.backward()
        
        # update the model parameters
        optimizer.step()
        
        # save the losses 
        train_loss[epoch] = error.item()
        
        # test loss 
        test_pred = model(x_test)
        test_error = criterion(y_test,test_pred)
        test_loss[epoch] = test_error.item()
        
        if (epoch+1) % 5 ==0:
            print('Epoch :{}    Train Loss :{}    Test Loss :{}'.format((epoch+1)/epochs, error.item(), test_error.item()))
            
    return train_loss, test_loss

In [17]:
train_loss, test_loss = train_model(model,
               loss_fun,
               optimizer,
               train_x,
               test_x,
               train_y,
               test_y,
               epochs=500)

Epoch :0.01    Train Loss :0.22151802480220795    Test Loss :0.1859532594680786
Epoch :0.02    Train Loss :0.14113779366016388    Test Loss :0.11390868574380875
Epoch :0.03    Train Loss :0.08234763145446777    Test Loss :0.06322388350963593
Epoch :0.04    Train Loss :0.04405637085437775    Test Loss :0.03219597041606903
Epoch :0.05    Train Loss :0.02295403927564621    Test Loss :0.016979124397039413
Epoch :0.06    Train Loss :0.014148196205496788    Test Loss :0.012364808470010757
Epoch :0.07    Train Loss :0.012338044121861458    Test Loss :0.013067294843494892
Epoch :0.08    Train Loss :0.013157417066395283    Test Loss :0.015043550170958042
Epoch :0.09    Train Loss :0.014019759371876717    Test Loss :0.01617300510406494
Epoch :0.1    Train Loss :0.014079054817557335    Test Loss :0.016030069440603256
Epoch :0.11    Train Loss :0.013545307330787182    Test Loss :0.01510421559214592
Epoch :0.12    Train Loss :0.012913322076201439    Test Loss :0.014038297347724438
Epoch :0.13    Tr

In [18]:
import plotly.graph_objects as go
trace11 = go.Scatter(
    x = np.arange(0, epochs,1),
    y = train_loss,
    mode = 'lines',
    name = 'Train Loss'
)
trace21 = go.Scatter(
    x = np.arange(0, epochs,1),
    y = test_loss,
    mode = 'lines',
    name = 'Test Loss'
)

layout = go.Layout(
    title = f"RNN performace",
    xaxis = {'title' : 'Epochs'},
    yaxis = {'title' : 'Loss'}
)
fig = go.Figure(data=[trace11, trace21], layout=layout)
fig.show()

In [19]:
# testing the prediction quality if the trained model

trace31 = go.Scatter(
    y = np.vstack((train_y.detach().numpy(),test_y.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Target'
)
trace41 = go.Scatter(
    y = np.vstack((train_y.detach().numpy(),model(test_x).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "RNN",
    xaxis = {'title' : 'Time delta in days'},
    yaxis = {'title' : f'{target}'}
)
fig = go.Figure(data=[trace31, trace41], layout=layout)
fig.show()

### LSTM

In [20]:
# Implement LSTM
class LSTM(nn.Module):
    def __init__(self,in_dim, hid_dim, out_dim, num_layers):
        super().__init__()
        self.in_dim = in_dim
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        self.layer = num_layers       
        self.lstm = nn.LSTM(self.in_dim, self.hid_dim, self.layer, batch_first=True)        
        self.fc = nn.Linear(self.hid_dim, self.out_dim)
        
    def forward(self,x):        
        # initialize the hidden state
        h0 = torch.zeros(self.layer, x.size(0), self.hid_dim)
        c0 = torch.zeros(self.layer, x.size(0), self.hid_dim)
        
        # forward propagate
        out, (h_,c_) = self.lstm(x,(h0,c0))
        
        out = self.fc(out[:,-1,:])
        return out

In [21]:
x, y = create_data(data,10)
# convert the x and y values into arrays
x, y = np.asarray(x), np.asarray(y)
x.reshape(-1,10).shape

(2440, 10)

In [22]:
# Define LSTM RNN neural network
in_size = x.shape[-1]
out_size = in_size
hid_size = 5
num_layers = 1
lstm = LSTM(in_size,hid_size,out_size, num_layers)
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.01)
loss_fun = nn.MSELoss()

In [23]:
# convert the data from numpy to tensor
train_x1, train_y1, test_x1, test_y1 = split_data(x, y, 0.7)
train_x1 = torch.from_numpy(train_x1.astype(np.float32))
train_y1 = torch.from_numpy(train_y1.astype(np.float32))
test_x1 = torch.from_numpy(test_x1.astype(np.float32))
test_y1 = torch.from_numpy(test_y1.astype(np.float32))

In [24]:
x.shape

(2440, 10, 1)

In [25]:
epochs1 = 500
train_loss1, test_loss1 = train_model(lstm,
               loss_fun,
               optimizer,
               train_x1,
               test_x1,
               train_y1,
               test_y1,
               epochs=epochs1)

Epoch :0.01    Train Loss :0.4352535903453827    Test Loss :0.3667069375514984
Epoch :0.02    Train Loss :0.2285323441028595    Test Loss :0.17117425799369812
Epoch :0.03    Train Loss :0.05790290609002113    Test Loss :0.031192190945148468
Epoch :0.04    Train Loss :0.014761069789528847    Test Loss :0.025669481605291367
Epoch :0.05    Train Loss :0.03738133981823921    Test Loss :0.040087487548589706
Epoch :0.06    Train Loss :0.017885353416204453    Test Loss :0.0175090953707695
Epoch :0.07    Train Loss :0.011460618115961552    Test Loss :0.01132195070385933
Epoch :0.08    Train Loss :0.014021983370184898    Test Loss :0.012574409134685993
Epoch :0.09    Train Loss :0.014003578573465347    Test Loss :0.012131902389228344
Epoch :0.1    Train Loss :0.0120155094191432    Test Loss :0.011331351473927498
Epoch :0.11    Train Loss :0.011419537477195263    Test Loss :0.012135119177401066
Epoch :0.12    Train Loss :0.011811826378107071    Test Loss :0.012912008911371231
Epoch :0.13    Trai

In [26]:
import plotly.graph_objects as go
trace12 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = train_loss1,
    mode = 'lines',
    name = 'Train Loss'
)
trace22 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = test_loss1,
    mode = 'lines',
    name = 'Test Loss'
)

layout = go.Layout(
    title = f"LSTM performace",
    xaxis = {'title' : 'Epochs'},
    yaxis = {'title' : 'Loss'}
)
fig = go.Figure(data=[trace12, trace22], layout=layout)
fig.show()

In [27]:
# testing the prediction quality if the trained model

trace32 = go.Scatter(
    y = np.vstack((train_y1.detach().numpy(),test_y1.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Target'
)
trace42 = go.Scatter(
    y = np.vstack((train_y1.detach().numpy(),lstm(test_x1).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "LSTM",
    xaxis = {'title' : 'Time delta in days'},
    yaxis = {'title' : f'{target}'}
)
fig = go.Figure(data=[trace32, trace42], layout=layout)
fig.show()

### Multi-layer gated recurrent unit RNN -  GRU

In [28]:
# implement GRU
class GRU(nn.Module):
    def __init__(self,in_dim, hid_dim, out_dim, num_layer):
        super().__init__()
        
        self.in_dim = in_dim
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        self.layer = num_layers
        
        self.gru = nn.GRU(self.in_dim, self.hid_dim, self.layer, batch_first=True)
        self.fc = nn.Linear(self.hid_dim, self.out_dim)
        
    def forward(self,x):
        
        # initalize the hidden layers
        h0 = torch.zeros(self.layer, x.size(0), self.hid_dim)
        
        out, _ = self.gru(x,h0)
        out = self.fc(out[:,-1,:])
        return out

In [29]:
# convert the data from numpy to tensor
train_x2, train_y2, test_x2, test_y2 = split_data(x, y, 0.7)
train_x2 = torch.from_numpy(train_x2.astype(np.float32))
train_y2 = torch.from_numpy(train_y2.astype(np.float32))
test_x2 = torch.from_numpy(test_x2.astype(np.float32))
test_y2 = torch.from_numpy(test_y2.astype(np.float32))

In [30]:
# definelGRU neurak network
in_size = x.shape[-1]
out_size = in_size
hid_size = 5
num_layers = 1

In [31]:
#__________ GRU Model__________________________________
gru = GRU(in_size,hid_size,out_size, num_layers)
optimizer = torch.optim.Adam(gru.parameters(), lr=0.01)
loss_fun = nn.MSELoss()

In [32]:
epochs2=500
train_loss2, test_loss2 = train_model(gru,
               loss_fun,
               optimizer,
               train_x2,
               test_x2,
               train_y2,
               test_y2,
               epochs=epochs2)

Epoch :0.01    Train Loss :0.06660772114992142    Test Loss :0.030884813517332077
Epoch :0.02    Train Loss :0.0205182246863842    Test Loss :0.034686099737882614
Epoch :0.03    Train Loss :0.043473999947309494    Test Loss :0.045684993267059326
Epoch :0.04    Train Loss :0.017509611323475838    Test Loss :0.016201462596654892
Epoch :0.05    Train Loss :0.01333459559828043    Test Loss :0.012783786281943321
Epoch :0.06    Train Loss :0.01801830530166626    Test Loss :0.014839776791632175
Epoch :0.07    Train Loss :0.01372868474572897    Test Loss :0.01172882504761219
Epoch :0.08    Train Loss :0.011911531910300255    Test Loss :0.013146850280463696
Epoch :0.09    Train Loss :0.013108639046549797    Test Loss :0.01458054967224598
Epoch :0.1    Train Loss :0.012103987857699394    Test Loss :0.012553434818983078
Epoch :0.11    Train Loss :0.011805921792984009    Test Loss :0.011536532081663609
Epoch :0.12    Train Loss :0.012021468952298164    Test Loss :0.011485489085316658
Epoch :0.13  

In [33]:
trace13 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = train_loss2,
    mode = 'lines',
    name = 'Train Loss'
)
trace23 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = test_loss2,
    mode = 'lines',
    name = 'Test Loss'
)

layout = go.Layout(
    title = f"RNN performace with GRU model",
    xaxis = {'title' : 'Epochs'},
    yaxis = {'title' : 'Loss'}
)
fig = go.Figure(data=[trace13, trace23], layout=layout)
fig.show()

In [34]:
# testing the prediction quality if the trained model

trace33 = go.Scatter(
    y = np.vstack((train_y2.detach().numpy(),test_y2.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Target'
)
trace43 = go.Scatter(
    y = np.vstack((train_y2.detach().numpy(),gru(test_x2).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "Multi-layer gated recurrent unit RNN - GRU",
    xaxis = {'title' : 'Time delta in days'},
    yaxis = {'title' : f'{target}'}
)
fig = go.Figure(data=[trace33, trace43], layout=layout)
fig.show()

### Autoregressive Model

In [35]:
# convert the data from numpy to tensor
train_x3, train_y3, test_x3, test_y3 = split_data(x, y, 0.7)
train_x3 = torch.from_numpy(train_x3.astype(np.float32))
train_y3 = torch.from_numpy(train_y3.astype(np.float32))
test_x3 = torch.from_numpy(test_x3.astype(np.float32))
test_y3 = torch.from_numpy(test_y3.astype(np.float32))

In [36]:
# Implement Autoregressive model
auto_reg = nn.Linear(10,1)
optimizer = torch.optim.Adam(auto_reg.parameters(), lr=0.01)
loss_fun = nn.MSELoss()

In [37]:
epochs3=500
train_loss3, test_loss3 = train_model(auto_reg,
               loss_fun,
               optimizer,
               train_x3.reshape(-1,10),
               test_x3.reshape(-1,10),
               train_y3.reshape(-1,1),
               test_y3.reshape(-1,1),
               epochs=epochs3)

Epoch :0.01    Train Loss :0.012702370062470436    Test Loss :0.012037084437906742
Epoch :0.02    Train Loss :0.021340571343898773    Test Loss :0.014897974207997322
Epoch :0.03    Train Loss :0.012393445707857609    Test Loss :0.009612162597477436
Epoch :0.04    Train Loss :0.01550803892314434    Test Loss :0.011424210853874683
Epoch :0.05    Train Loss :0.012244376353919506    Test Loss :0.008927538990974426
Epoch :0.06    Train Loss :0.013322088867425919    Test Loss :0.009797312319278717
Epoch :0.07    Train Loss :0.012119457125663757    Test Loss :0.008816122077405453
Epoch :0.08    Train Loss :0.012459793128073215    Test Loss :0.009128786623477936
Epoch :0.09    Train Loss :0.011962201446294785    Test Loss :0.00875253975391388
Epoch :0.1    Train Loss :0.01204700581729412    Test Loss :0.008820491842925549
Epoch :0.11    Train Loss :0.011816907674074173    Test Loss :0.008692312985658646
Epoch :0.12    Train Loss :0.011785551905632019    Test Loss :0.008661148138344288
Epoch :0

In [38]:
trace14 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = train_loss3,
    mode = 'lines',
    name = 'Train Loss'
)
trace24 = go.Scatter(
    x = np.arange(0, epochs1,1),
    y = test_loss3,
    mode = 'lines',
    name = 'Test Loss'
)

layout = go.Layout(
    title = f"RNN performace with Autorgressive model",
    xaxis = {'title' : 'Epochs'},
    yaxis = {'title' : 'Loss'}
)
fig = go.Figure(data=[trace14, trace24], layout=layout)
fig.show()

In [39]:
# testing the prediction quality if the trained model

trace34 = go.Scatter(
    y = np.vstack((train_y3.detach().numpy(),test_y3.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Target'
)
trace44 = go.Scatter(
    y = np.vstack((train_y3.detach().numpy(),auto_reg(test_x3.reshape(-1,10)).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "RNN performace with Autorgressive model",
    xaxis = {'title' : 'Time delta in days'},
    yaxis = {'title' : f'{target}'}
)
fig = go.Figure(data=[trace34, trace44], layout=layout)
fig.show()

## Comparing all models

In [40]:
pred_NN = model(train_x)
pred_gru = gru(train_x)
pred_lstm = lstm(train_x)
#pred_AR = auto_reg(train_x)

In [41]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
r2_NN = r2_score(train_y, pred_NN.detach().numpy() )
r2_gru = r2_score(train_y, pred_gru.detach().numpy() )
r2_lstm = r2_score(train_y, pred_lstm.detach().numpy() )
mse_NN = mean_squared_error(train_y, pred_NN.detach().numpy(), squared=False)
mse_gru = mean_squared_error(train_y, pred_gru.detach().numpy(), squared=False)
mse_lstm = mean_squared_error(train_y, pred_lstm.detach().numpy(), squared=False)

print(f'Root Mean Squared Error NN = {mse_NN} r2_NN = {r2_NN}')
print(f'Root Mean Squared Error GRU = {mse_gru} r2_GRU = {r2_gru}')
print(f'Root Mean Squared Error LSTM = {mse_lstm} r2_LSTM = {r2_lstm}')

Root Mean Squared Error NN = 0.09993283450603485 r2_NN = 0.19694757294108578
Root Mean Squared Error GRU = 0.09716758131980896 r2_GRU = 0.24077539119629765
Root Mean Squared Error LSTM = 0.098174087703228 r2_LSTM = 0.224965038263522


In [42]:
# testing the prediction for all models

trace15 = go.Scatter(
    y = np.vstack((train_y.detach().numpy(),test_y.detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'markers',
    name = 'Training data'
)
trace25 = go.Scatter(
    y = np.vstack((train_y.detach().numpy(),model(test_x).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    opacity=1,
    name = f'Prediction with simple RNN (rsme={mse_NN:.3f}, r2={r2_NN:.3f})'
)

trace35 = go.Scatter(
    y = np.vstack((train_y1.detach().numpy(),lstm(test_x1).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    opacity=0.7,
    mode = 'lines',
    name = f'Prediction with LSTM model (rsme={mse_lstm:.3f}, r2={r2_lstm:.3f})'
)

trace45 = go.Scatter(
    y = np.vstack((train_y2.detach().numpy(),gru(test_x2).detach().numpy())).reshape(-1),
    x = np.arange(0, len(y), 1),
    mode = 'lines',
    opacity=0.7,
    name = f'Prediction with GRU model (rsme={mse_gru:.3f}, r2={r2_gru:.3f})'
)

#trace55 = go.Scatter(
#    y = np.vstack((train_y3.detach().numpy(), auto_reg(test_x3.reshape(-1,10)).detach().numpy())).reshape(-1),
#    x = np.arange(0, len(y), 1),
#    mode = 'lines',
#)
#    name = 'Prediction with Autoregressive Model'
#layout = go.Layout(
#    title = f"Evolution of feature '{target}' with time",
#    xaxis = {'title' : 'Time delta in days'},
#    yaxis = {'title' : f' Feature = {target}'}
#)
fig = go.Figure(data=[trace15, trace25, trace35, trace45], layout=layout)
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
    plot_bgcolor='rgba(0,0,0,0)',
    title = "Performace with different RNN models"
    )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_xaxes(showline=True, linewidth=1, linecolor='grey',gridcolor='grey')
fig.update_yaxes(showline=True, linewidth=1, linecolor='grey', gridcolor='grey')
fig.show()

In [55]:
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn import metrics
from sklearn import metrics

#plot_roc_curve(model,test_x, test_y)
fpr, tpr, thresholds = metrics.roc_curve(test_y.detach().numpy(),pred_NN.detach().numpy(), pos_label=2)
metrics.auc(fpr, tpr)

ValueError: ignored